In [ ]:
import os
import shutil
import subprocess
from datetime import datetime
import warnings

In [ ]:
# TODOs:
# - crawl in directory provided (this will make it difficult for mdls command)
# - handle folders with many files...
# - MOV files need to be associated with their image files :( they are created at export time! FUCK
# - sort / isolate duplicates

In [ ]:
# set root folder:
folder = "/Volumes/EN480G/preProcessed/2020_01_19_export/"
# generate list of files with full paths:
flist = []
for (dirpath, dirnames, filenames) in os.walk(folder):
    # for each file:
    for f in filenames:
        fpath = os.path.join(dirpath, f)
        # ignore hidden files:
        if f[0] != '.':
            flist.append(fpath)

In [ ]:
# for each file in its path, see if its got a matching one (or more)
flistGrouped = []
_tempflist = flist.copy()

while _tempflist:
    # pop first item out of _tempflist:
    _file = _tempflist.pop(0)
    _fname = _file.split('/')[-1]
    _fnameBase = _fname.split('.')[0]
    group = [_file]
    k = 0
    while k != len(_tempflist):
        _testFile = _tempflist[k]
        _testfileName = _testFile.split('/')[-1]
        _testfileNameBase = _testfileName.split('.')[0]
        # if tested file matched _fnameBase, pop it, dont update k:
        if _fnameBase == _testfileNameBase:
            group.append(_testFile)
            _tempflist.pop(k)
        else:
            k = k + 1
    flistGrouped.append(group)

In [ ]:
# At this point, flistGrouped is a set of images grouped by their names.
# each index of listGrouped with 1 entry can be processed as usual, only need to look out for 
# jan 1 1970 create date
singleInstances = []
groupInstances = []
for _item in flistGrouped:
    if len(_item)==1:
        singleInstances.append(_item)
    else:
        groupInstances.append(_item)
            

In [ ]:
"""
For each item in single instance, get mdls data...
"""
singleMapping = []
singleMapping_undetermined = []
for _item in singleInstances:
    _name = _item[0]
    # make the filename mdls query friendly...
    qfile = _name
    cmd = "mdls"\
          " -name kMDItemContentCreationDate" \
          " -name kMDItemContentModificationDate" \
          " -name kMDItemFSContentChangeDate" \
          " -name kMDItemFSCreationDate" \
          " '{:s}'".format(qfile)
    ret = subprocess.run(cmd,capture_output=True, shell=True)
    # check for errors:
    err = ret.stderr.decode('utf-8').split("\n")
    if not err[0] == '':
        raise Exception(err)
    # parse "ret":
    pOut = ret.stdout.decode('utf-8').split("\n")
    # on mac; if any has "null" need to reindex drive using `sudo mdutil -E /Volume`
    # get dates:
    d_created = datetime.strptime(pOut[0].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    d_createdMod = datetime.strptime(pOut[1].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    d_FileChanged = datetime.strptime(pOut[2].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    d_FileCreated = datetime.strptime(pOut[3].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')

    if d_FileCreated.year == 1970:
        # need to ignore this...
        pass

    # create object:
    fObj = {}
    fObj['filePath'] = _name
    fObj['ext'] = _name.split('.')[-1]
    fObj['dstName'] = d_created.strftime("%Y_%m_%d-%H_%M_%S%z")
    fObj['dCC'] = d_created;
    fObj['dCM'] = d_createdMod;
    fObj['dFC'] = d_FileCreated;
    fObj['dFCC'] = d_FileChanged;
    singleMapping.append(fObj)

# create date object with specific time:
_dl = datetime.strptime("2020-01-15_23-00 +0000", '%Y-%m-%d_%H-%M %z')
_du = datetime.strptime("2020-01-15_23-59 +0000", '%Y-%m-%d_%H-%M %z')

In [ ]:
def getMDLSDatInfo(fileName):
    cmd = "mdls"\
          " -name kMDItemContentCreationDate" \
          " -name kMDItemContentModificationDate" \
          " -name kMDItemFSContentChangeDate" \
          " -name kMDItemFSCreationDate" \
          " '{:s}'".format(fileName)
    ret = subprocess.run(cmd,capture_output=True, shell=True)
    # check for errors:
    err = ret.stderr.decode('utf-8').split("\n")
    if not err[0] == '':
        raise Exception(err)
    # parse "ret":
    pOut = ret.stdout.decode('utf-8').split("\n")
    output = {}
    # get dates:
    output['created'] = datetime.strptime(pOut[0].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['modified'] = datetime.strptime(pOut[1].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['file_changed'] = datetime.strptime(pOut[2].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['file_created'] = datetime.strptime(pOut[3].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    
    return output

In [ ]:
"""
For each item in group instance, get mdls data...
"""
groupMapping = []
for _items in groupInstances:
    # should have one '.mov' and another not
    _movFile = [i for i in _items if i.endswith('.mov')][0]
    _imgFile = [i for i in _items if not i.endswith('.mov')][0]
    
    # get image date info:
    _imgDateInfo = getMDLSDatInfo(_imgFile)
    # get MOV date info:
    _movDateInfo = getMDLSDatInfo(_imgFile)
    # create Img object:
    fObj = {}
    fObj['filePath'] = _imgFile
    fObj['ext'] = _imgFile.split('.')[-1]
    fObj['dstName'] = _imgDateInfo['created'].strftime("%Y_%m_%d-%H_%M_%S%z")
    
    fObjmov = {}
    fObjmov['filePath'] = _movFile
    fObjmov['ext'] = _movFile.split('.')[-1]
    # check if MOV date created is newer than img:
    if _movDateInfo['created'] > _imgDateInfo['created']:
        # set movie filename destdate to that of img
        fObjmov['dstName'] = _imgDateInfo['created'].strftime("%Y_%m_%d-%H_%M_%S%z")
    

In [ ]:
folder = "/Volumes/EN480G/preProcessed/2020_01_19_export/"
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(folder):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]

In [ ]:

folder_out = "/Volumes/EN480G/postProcessed/"
files = os.listdir(folder)
# get extensions:
exts = set([f.split(".")[-1] for f in files])
print(exts)
ext_blacklist = ['db', 'ini', 'DS_Store', '_JUTCA~0', 'I7CIPB~N', 'LRV', 'THM', 'AAE']
# since using shell to run command, need to make single query string:
cmd = "mdls -name kMDItemContentCreationDate \
       -name _kMDItemDisplayNameWithExtensions \
       {:s}*".format(folder)

In [ ]:
# run the command:
ret = subprocess.run(cmd,capture_output=True, shell=True)
# check for errors:
err = ret.stderr.decode('utf-8').split("\n")
if not err[0] == '':
    raise Exception(err)

In [ ]:
# decode the return byte string:
ff = ret.stdout.decode('utf-8').split("\n")

In [ ]:
fInfo = {}
for k in range(0,len(ff)-1,2):
    # file name in folder:
    nn = ff[k].split("=")[1].replace('"', '').lstrip()
    dd = datetime.strptime(ff[k+1].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    # only grab files whose extensions are not in the black list
    if not nn.split(".")[-1] in ext_blacklist:
        fInfo[nn] = dd


In [ ]:
for im in fInfo:
    # move original from src to dst with new datestring
    ext = im.split(".")[-1]
    src = folder + im
    dfile = fInfo[im].strftime("%Y_%m_%d-%H_%M_%S%z") + "." + ext
    dfolder = folder_out + str(fInfo[im].year) +"/"
    if not os.path.isdir(dfolder):
        os.mkdir(dfolder)
    dst = dfolder + dfile
    # verify correctness..
    if not os.path.isfile(src):
        raise Exception("source file {:s} doesnt exist!\n".format(src))
    # check if dst file exists
    duplicate = os.path.isfile(dst)
    ii = 1
    #if dst file exists, add postfix to file with increment
    while (duplicate):
        warnings.warn("desination file {:s} EXISTS from source: {:s}!\n".format(dst, src))
        dst = dfolder + fInfo[im].strftime("%Y_%m_%d-%H_%M_%S%z") + "_{:03d}.".format(ii) + ext
        duplicate = os.path.isfile(dst)
        ii += 1
    # do move
    shutil.move(src,dst)